In [16]:
from osgeo import gdal
import leafmap

In [17]:
# Localizar os arquivos

red_band_path = r''
nir_band_path = r''

In [18]:
# Abrir os arquivos

red_band_ds = gdal.Open(red_band_path)
nir_band_ds = gdal.Open(nir_band_path)

In [19]:
# Transformar em Array

red_band_data = red_band_ds.ReadAsArray()
nir_band_data = nir_band_ds.ReadAsArray()

In [20]:
# Calcula o NDVI

ndvi = (nir_band_data - red_band_data) / (nir_band_data + red_band_data)

In [21]:
# Adiciona o caminho do arquivo e as informações necessárias para criar o raster

output_ndvi_path = r''

width = red_band_ds.RasterXSize # Largura do Raster
height = red_band_ds.RasterYSize # Altura do Raster
projection = red_band_ds.GetProjection() # Sistema de Coordenadas
geotransform = red_band_ds.GetGeoTransform() # Matriz de Transformação
bands = red_band_ds.RasterCount # Número de bandas

In [22]:
# Cria o raster de saída

driver = gdal.GetDriverByName('Gtiff')

ndvi_ds = driver.Create(output_ndvi_path, width, height, bands, gdal.GDT_Float32)

In [23]:
ndvi_ds.SetGeoTransform(geotransform)
ndvi_ds.SetProjection(projection)

0

In [24]:
# Escreve o resultado da aritmética de bandas no raster criado

ndvi_band = ndvi_ds.GetRasterBand(1)
ndvi_band.WriteArray(ndvi)

0

In [25]:
# Calcula as estatísticas do raster

ndvi_band.ComputeStatistics(True)

[-1.0, 1.0, 0.1845122403193951, 0.1844916223602398]

In [26]:
# Limpa as variáveis e libera memória

red_band_ds = None
nir_band_ds = None
ndvi_ds = None

In [27]:
m = leafmap.Map(center=[-22,-51], zoom=18, width='800px')
m.add_basemap("SATELLITE")
m

Map(center=[-22, -51], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [28]:
m.layers[-1].visible = True
m.add_raster(output_ndvi_path,scheme='JenksCaspall',cmap='Greens',legend_title='NDVI')

In [29]:
import rasterio
import numpy as np

with rasterio.open(output_ndvi_path) as src:
    ndvi = src.read(1)
    print("NDVI - mínimo:", np.nanmin(ndvi))
    print("NDVI - máximo:", np.nanmax(ndvi))
    print("NDVI - média:", np.nanmean(ndvi))

NDVI - mínimo: -1.0
NDVI - máximo: 1.0
NDVI - média: 0.18496847
